In [ ]:
"""
note
2025.03.13
미니 프로젝트 시작.
상관계수를 직접 구해보는 과정을 해보고 싶었으나 아직 통계와 분석에 대한 이해가 떨어져 바로 결정트리 구현을 진행함.
2025.03.14
넘파이 없이 구현을 목표로 했으나 특징벡터 4개, 클래스 3개로 계산량이 너무 커서 넘파이를 사용해 구현,
추후 상관계수 계산 후 특징벡터 2개를 이용한 이진, 혹은 3 클래스 분류를 진행하고자함.
2025.03.19
pandas 사용을 하지 않으려 했으나 numpy 만을 사용 할 경우 한 배열 당 하나의 데이터 타입만 지정 가능한 조건에서 어려움이 발생함.
이를 dict, list 를 이용해 일부 해결이 가능하나 pandas를 사용하기로 결정함.
"""

'\nnote\n2025.03.13\n미니 프로젝트 시작.\n상관계수를 직접 구해보는 과정을 해보고 싶었으나 아직 통계와 분석에 대한 이해가 떨어져 바로 결정트리 구현을 진행함.\n2025.03.14\n넘파이 없이 구현을 목표로 했으나 특징벡터 4개, 클래스 3개로 계산량이 너무 커서 넘파이를 사용해 구현,\n추후 상관계수 계산 후 특징벡터 2개를 이용한 이진, 혹은 3 클래스 분류를 진행하고자함.\n'

In [1]:
import numpy as np
import pandas as pd

np.random.seed(0)

In [2]:
df = pd.read_csv('Iris.csv', index_col=0)
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
Id,,,,,
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
df.columns = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'target']
df['target'] = df['target'].map({'Iris-versicolor':0, 'Iris-setosa':1, 'Iris-virginica':2})

In [5]:
df

,SepalLength,SepalWidth,PetalLength,PetalWidth,target
Id,,,,,
1,5.1,3.5,1.4,0.2,1
2,4.9,3.0,1.4,0.2,1
3,4.7,3.2,1.3,0.2,1
4,4.6,3.1,1.5,0.2,1
5,5.0,3.6,1.4,0.2,1
...,...,...,...,...,...
146,6.7,3.0,5.2,2.3,2
147,6.3,2.5,5.0,1.9,2
148,6.5,3.0,5.2,2.0,2


In [6]:
# 데이터의 클래스 비율을 유지 할 수 있었으나 이후 랜덤포레스트 구현 시 발생 할 차이를 보기 위해 랜덤으로 유지함.

def train_test_split(df, ratio=0.7, split_target=False, target_label='target'):
    X_train = df.sample(frac=ratio)
    X_test = df.drop(X_train.index)
    
    if (split_target):
        Y_train = X_train.pop(target_label).to_frame()
        Y_test = X_test.pop(target_label).to_frame()
        X_train, X_test, Y_train, Y_test = [_df.reset_index(drop=True) for _df in [X_train, X_test, Y_train, Y_test]]
        return X_train, X_test, Y_train, Y_test
    
    X_train, X_test = [_df.reset_index() for _df in [X_train, X_test]]
    return X_train, X_test

In [7]:
def min_max_scaler(x, _skipna=True, _numeric_only=True):
    _min = x.min(axis=0, skipna=_skipna, numeric_only=_numeric_only)
    _max = x.max(axis=0, skipna=_skipna, numeric_only=_numeric_only)
    return x.sub(_min).truediv(_max-_min)

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(df, split_target=True)
X_train = min_max_scaler(X_train)
X_test = min_max_scaler(X_test)
len(X_train), len(X_test), len(Y_train), len(Y_test)

(105, 45, 105, 45)

In [9]:
X_train.iloc[[25, 69, 67, 80, 3, 104]]

,SepalLength,SepalWidth,PetalLength,PetalWidth
25,0.088235,0.666667,0.000000,0.041667
69,0.000000,0.416667,0.017857,0.000000
67,0.852941,0.500000,0.892857,0.708333
80,0.852941,0.666667,0.910714,1.000000
3,0.882353,0.375000,0.946429,0.708333
104,0.970588,0.416667,1.000000,0.833333


In [440]:
"""
결정트리분류 모델 구현
손실함수가 필요 없는 모델이며 지니지수를 이용해 데이터를 분류한다.
분류에는 2~4 특징을 이용한다. 성능 평가는 ACC(정확도)를 사용한다.
연결리스트, 노드를 사용한다.
attribute:
DB = Decision_Boundary(결정경계) - 모델로 구하고자 하는 목표, 데이터를 분류하는 기준.
maxG = maxG가 작아지도록 결정경계를 설정한다. 기존값은 1. 결정경계를 생성하면서 점점 낮아진다.

DH = Decision_History - 트리가 클래스를 분류한 기록
"""

'\n결정트리분류 모델 구현\n손실함수가 필요 없는 모델이며 지니지수를 이용해 데이터를 분류한다.\n분류에는 2~4 특징을 이용한다. 성능 평가는 ACC(정확도)를 사용한다.\n연결리스트, 노드를 사용한다.\nattribute:\nDB = Decision_Boundary(결정경계) - 모델로 구하고자 하는 목표, 데이터를 분류하는 기준.\nmaxG = maxG가 작아지도록 결정경계를 설정한다. 기존값은 1. 결정경계를 생성하면서 점점 낮아진다.\n\nDH = Decision_History - 트리가 클래스를 분류한 기록\n'

In [13]:
for data in [X_train, X_test, Y_train, Y_test]:
    print(data.head(3))

   SepalLength  SepalWidth  PetalLength  PetalWidth
0     0.441176    0.333333     0.732143    0.958333
1     0.500000    0.083333     0.535714    0.375000
2     0.352941    0.916667     0.071429    0.041667
   SepalLength  SepalWidth  PetalLength  PetalWidth
0     0.151515    0.666667     0.035088    0.043478
1     0.090909    0.444444     0.052632    0.000000
2     0.363636    0.944444     0.000000    0.043478
   target
0       2
1       0
2       1
   target
0       1
1       1
2       1


In [51]:
# 1 - np.sum((Y_train.value_counts()/len(Y_train))**2)
class DecisionTreeClassifier:
    def __init__(self, depth = 2):
        self.depth = depth                  # 트리의 깊이 = 분할 횟수

        self.DB = [() for _ in range(2**depth-1)]

        self.data = None
        self.target = None
        self.pMap = None
        

    # 서브트리 루트 노드를 받아 최적의 결정경계 산출
    # 입력으로 루트 노드의 데이터 인덱스(Id)를 받음

    def compute1(self, _sorted, _cnt, _idx):       # Gini 불순도 계산 함수

        _left = np.bincount(_sorted[:_idx]['target'])
        _right = np.bincount(_sorted[_idx:]['target'])

        _GLeft = 1 - np.sum((_left / _idx)**2)
        _GRight = 1 - np.sum((_right/(_cnt -_idx))**2)

        return ( _idx *_GLeft + (_cnt -_idx) * _GRight ) /_cnt
        
    def CART(self, _id, _pos):
        _x = self.data.iloc[_id]       # df로 저장
        _y = self.target.iloc[_id]     # df로 저장
        
        print(f'_pos:{_pos}, _id cnt :{len(_id)}', np.bincount(_y['target']))

        _cnt = len(_id)
        _db = None
        _maxG = 1 - np.sum((_y['target'].value_counts()/_cnt)**2)          # 데이터를 나누지 않았을 때 불순도
        
        for _col in _x.columns:
            print(_y.index)
            print(_x.index)
            print(_x[[_col]].sort_values(by=_col).index)
            _sorted = _y[_x[[_col]].sort_values(by=_col).index]
        
            for _idx in range(1, _x.shape[0]-1):
                _G = self.compute1(_sorted, _cnt, _idx)
                if (_G < _maxG):
                    _maxG = _G
                    _db = (_col, _idx)

        if _db is None:                                             # 불순도를 더 줄이는 결정경계가 없을 때
            self.pMap[_pos] = np.argmax(np.bincount(_y['target']))
            return
        
        _idxDB = 0
        for i in _pos:
            _idxDB = 2 *_idxDB +1 +i
        f, _idx = _db

        _sorted = _x[[f]].sort_values(by=f)
        _left = _sorted.iloc[:_idx].index.to_numpy()
        _right = _sorted.iloc[_idx:].index.to_numpy()

        if len(_pos) == self.depth-1:                               # 최고 깊이 도달
            self.DB[_idxDB] = f, np.mean(_sorted.iloc[_idx-1:_idx+1])
            self.pMap[_pos+(0,)] = np.argmax(np.bincount(_y['target'][_left]))
            self.pMap[_pos+(1,)] = np.argmax(np.bincount(_y['target'][_right]))

        else:
            self.DB[_idxDB] = f, np.mean(_sorted.iloc[_idx-1:_idx+1])
            self.CART(_left,  _pos+(0,))
            self.CART(_right, _pos+(1,))

    def fit(self, x, y):
        self.data = x
        self.target = y
        self.pMap = np.zeros((2,)*self.depth)    # 에측 결과 대응 맵
        # CART 의 파라미터로 데이터프레임과 데이터 인덱스 어느쪽을 넘겨주는것이 좋은가?
        self.CART(np.arange(self.target.size), ())

    def predict(self, x):
        _pred = {}
        for s in x.iloc:
            _pos = ()
            _idx = 0
            for _ in range(self.depth):
                _pos += self.compute2(s, self.DB[_idx])
                _idx = 2 *_idx +1 +_pos[-1]
            if(_pos == ()):
                print("error error error error / _pos == () / error error error error")
                return
            _pred[f'{s.name}'] = [self.pMap[_pos]]
        return _pred

    def compute2(self, s, f_v):
        if f_v == ():
            return (0,) # 트리 조기종료시 결과는 동일함.
        f, v = f_v
        if s[f] > v:
            return (0,)
        else:
            return (1,)

In [52]:
dtc = DecisionTreeClassifier(3)
dtc.fit(X_train, Y_train)

_pos:(), _id cnt :105 [34 33 38]
Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
       ...
        95,  96,  97,  98,  99, 100, 101, 102, 103, 104],
      dtype='int64', length=105)
Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
       ...
        95,  96,  97,  98,  99, 100, 101, 102, 103, 104],
      dtype='int64', length=105)
Index([ 69,  23,  96, 100,  91,  75,  78,  25,  41,  82,
       ...
        95,  84,  58,  83,  67,  80,  98,   3, 104,  51],
      dtype='int64', length=105)


KeyError: "None of [Index([ 69,  23,  96, 100,  91,  75,  78,  25,  41,  82,\n       ...\n        95,  84,  58,  83,  67,  80,  98,   3, 104,  51],\n      dtype='int64', length=105)] are in the [columns]"

In [492]:
dtc.DB, dtc.pMap

([('PetalLength', 0.2589285714285714),
  ('PetalLength', 0.14285714285714285),
  ('PetalLength', 0.9285714285714286),
  ('PetalLength', 0.125),
  (),
  ('PetalLength', 0.9017857142857143),
  ()],
 array([[[1., 1.],
         [1., 1.]],
 
        [[0., 2.],
         [2., 2.]]]))

In [493]:
dtc.predict(X_test)

{'0': [2.0],
 '1': [2.0],
 '2': [2.0],
 '3': [2.0],
 '4': [2.0],
 '5': [2.0],
 '6': [2.0],
 '7': [2.0],
 '8': [2.0],
 '9': [2.0],
 '10': [2.0],
 '11': [2.0],
 '12': [2.0],
 '13': [2.0],
 '14': [2.0],
 '15': [2.0],
 '16': [2.0],
 '17': [1.0],
 '18': [1.0],
 '19': [1.0],
 '20': [1.0],
 '21': [1.0],
 '22': [1.0],
 '23': [1.0],
 '24': [1.0],
 '25': [1.0],
 '26': [1.0],
 '27': [1.0],
 '28': [1.0],
 '29': [1.0],
 '30': [1.0],
 '31': [1.0],
 '32': [1.0],
 '33': [1.0],
 '34': [1.0],
 '35': [1.0],
 '36': [1.0],
 '37': [1.0],
 '38': [1.0],
 '39': [1.0],
 '40': [1.0],
 '41': [1.0],
 '42': [1.0],
 '43': [1.0],
 '44': [1.0]}

In [19]:
np.bincount(np.array([0,4]))

array([1, 0, 0, 0, 1], dtype=int64)

In [494]:
_pred = pd.DataFrame(dtc.predict(X_test))
_pred.T

,0
0,2.0
1,2.0
2,2.0
3,2.0
4,2.0
5,2.0
6,2.0
7,2.0
8,2.0
9,2.0


In [498]:
Y_test

,target
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


In [401]:
tt = ()
tt[-1]

IndexError: tuple index out of range

In [ ]:
tt = {'0':0, '1':1, '2':2}
tt['0']

0

In [70]:
(0,)[1:]

()

In [2]:
[[] for _ in range(2**3-1)]

[[], [], [], [], [], [], []]

In [123]:
#tt = np.repeat([[[1],[None]]], 10, axis=0)
tt = np.arange(30).reshape(-1,2)
tt[0, 1] = 10
tt.shape[0]

15

In [150]:
_index = tt[:,0]==0
_index

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False])

In [186]:
list(range(tt.shape[0]))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [149]:
tid = [0,1,2,3]
tt[tid], tt

(array([[ 0, 10],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7]]),
 array([[ 0, 10],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7],
        [ 8,  9],
        [10, 11],
        [12, 13],
        [14, 15],
        [16, 17],
        [18, 19],
        [20, 21],
        [22, 23],
        [24, 25],
        [26, 27],
        [28, 29]]))

In [170]:
import numpy as np

# (5, 3) 형태의 NumPy 배열 (예제 데이터)
arr = np.array([
    [1, 2, 0],
    [2, 5, 1],
    [3, 1, 2],
    [4, 7, 0],
    [5, 6, 0],
    [6, 3, 1],
    [7, 2, 2],
    [8, 1, 0],
])

# 정렬 기준이 되는 (n,1) 열 선택 (예: 두 번째 열 기준으로 정렬)
column_index = 1  # 기준 열 인덱스
sorted_indices = np.argsort(arr[:, column_index])  # 기준 열을 정렬한 인덱스

# 정렬된 배열
sorted_arr = arr[sorted_indices]

print(sorted_arr)
arr1 = arr[np.argsort(arr[:, 1])][:, [1, -1]]

[[3 1 2]
 [8 1 0]
 [1 2 0]
 [7 2 2]
 [6 3 1]
 [2 5 1]
 [5 6 0]
 [4 7 0]]


In [171]:
arr1 = pd.DataFrame(arr1)
def ttest(arr1):
    arr1.drop(index = [1,2,3], inplace=True)
ttest(arr1)
arr1

,0,1
0,1,2
4,3,1
5,5,1
6,6,0
7,7,0


In [172]:
arr1[:2]

,0,1
0,1,2
4,3,1


In [24]:
t0 = np.bincount(arr1[:, -1])
for _idx in range(arr1.shape[0]):                             
    t1 = np.bincount(arr1[:_idx, -1], minlength=3) / t0
    t2 = np.bincount(arr1[_idx:, -1], minlength=3) / t0
np.add(t1**2, t2**2)

array([0.625, 1.   , 1.   ])

In [23]:
_idx = 2
np.bincount(arr[:,-1]), arr[:,-1]
arr[:_idx, -1].view(), arr[_idx:, -1].view()

(array([0, 1]), array([2, 0, 0, 1, 2, 0]))

In [32]:
tt0 = np.array([[1, 2, 3],[4, 5, 6]])
np.argmax(tt0)

5

In [247]:
t1 = 1; t2 = 2
int(t1 > t2)

0